In [ ]:
import typesense

## connect with the typesense database

In [ ]:
client = typesense.Client({
  'nodes': [{
    'host': 'localhost', 
    'port': '8108',     
    'protocol': 'http'
  }],
  'api_key': 'xyz',
  'connection_timeout_seconds': 100
})

## Prepare the data

In [102]:
import pandas as pd

df = pd.read_csv("products.csv").loc[:499]
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [105]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           500
Name: count, dtype: int64

In [104]:
df.fillna("None", inplace=True)

## Convert the relevant field to the Vector using BERT model

In [107]:
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-mpnet-base-v2")

In [86]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))


In [87]:
df.head()

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor,DescriptionVector
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black,"[0.027645761, -0.0026341833, -0.0035884224, 0...."
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige,"[-0.024660718, -0.02875545, -0.020332465, 0.03..."
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink,"[-0.046943277, 0.08182791, 0.0483352, -0.00015..."
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue,"[-0.015098808, -0.01028547, 0.00948734, -0.023..."
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White,"[-0.017746499, 0.0062095164, 0.021813968, 0.02..."


## Create new index in ElasticSearch

In [ ]:
mappings = {
    "name": "products",
    "fields": [
        {
            "name" : "ProductID",
            "type" : "int32"
        },
        {
            "name" : "ProductName",
            "type" : "string"
        },
        {
            "name" : "ProductBrand",
            "type" : "string"
        },
        {
            "name" : "Gender",
            "type" : "string"
        },
        {
            "name" : "Price (INR)",
            "type" : "int32"
        },
        {
            "name" : "NumImages",
            "type" : "int32"
        },
        {
            "name" : "Description",
            "type" : "string"
        },
        {
            "name" : "PrimaryColor",
            "type" : "string"
        },
        {
            "name" : "DescriptionVector",
            "type" : "float[]",
            "embed": {
              "from": [
                "Description"
              ],
              "model_config": {
                "model_name": "ts/all-MiniLM-L12-v2"
              }
            }
        }
    ]
}

client.collections.create(mappings)

{'created_at': 1743079355,
 'default_sorting_field': '',
 'enable_nested_fields': False,
 'fields': [{'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'ProductID',
   'optional': False,
   'sort': True,
   'stem': False,
   'stem_dictionary': '',
   'store': True,
   'type': 'int32'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'ProductName',
   'optional': False,
   'sort': False,
   'stem': False,
   'stem_dictionary': '',
   'store': True,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'ProductBrand',
   'optional': False,
   'sort': False,
   'stem': False,
   'stem_dictionary': '',
   'store': True,
   'type': 'string'},
  {'facet': False,
   'index': True,
   'infix': False,
   'locale': '',
   'name': 'Gender',
   'optional': False,
   'sort': False,
   'stem': False,
   'stem_dictionary': '',
   'store': True,
   'type': 'string'},
  {'facet': False

## Ingest the data into index

In [ ]:
records = df.to_dict("records")


{'ProductID': 10017413,
 'ProductName': 'DKNY Unisex Black & Grey Printed Medium Trolley Bag',
 'ProductBrand': 'DKNY',
 'Gender': 'Unisex',
 'Price (INR)': 11745,
 'NumImages': 7,
 'Description': 'Black and grey printed medium trolley bag, secured with a TSA lockOne handle on the top and one on the side, has a trolley with a retractable handle on the top and four corner mounted inline skate wheelsOne main zip compartment, zip lining, two compression straps with click clasps, one zip compartment on the flap with three zip pocketsWarranty: 5 yearsWarranty provided by Brand Owner / Manufacturer',
 'PrimaryColor': ' Black'}

In [ ]:


for record in records:
    try:
        client.collections["products"].documents.create(record)
    except Exception as e:
        print(e)

## Search the data

In [ ]:
input = "boy's t-shirt"

query = {
        "q": input,
        "query_by": "DescriptionVector",
        "exclude_fields": "DescriptionVector"
    }

res = client.collections['products'].documents.search(query)
result = res["hits"]
result

[{'document': {'Description': 'T-shirt',
   'Gender': 'Men',
   'NumImages': 5,
   'Price (INR)': 999,
   'PrimaryColor': ' Yellow',
   'ProductBrand': 'Raymond',
   'ProductID': 10016703,
   'ProductName': 'Raymond Men Yellow Solid Polo Collar T-shirt',
   'id': '412'},
  'highlight': {},
  'highlights': [],
  'vector_distance': 0.21688216924667358},
 {'document': {'Description': 'T-shirt',
   'Gender': 'Men',
   'NumImages': 5,
   'Price (INR)': 499,
   'PrimaryColor': ' Green',
   'ProductBrand': 'Difference of Opinion',
   'ProductID': 10005937,
   'ProductName': 'Difference of Opinion Men Olive Green Printed Round Neck T-shirt',
   'id': '128'},
  'highlight': {},
  'highlights': [],
  'vector_distance': 0.21688216924667358},
 {'document': {'Description': 'Green printed T-shirt, has a polo collar, and short sleeves',
   'Gender': 'Men',
   'NumImages': 5,
   'Price (INR)': 629,
   'PrimaryColor': ' Green',
   'ProductBrand': 'Parx',
   'ProductID': 10000245,
   'ProductName': 'Par